# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [5]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [65]:
!pip install geopandas
import geopandas as gpd

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (5.5 kB)
  Using cached pyproj-3.7.0.tar.gz (225 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      proj executable not found. Please set the PROJ_DIR variable. For more information see: https://pyproj4.github.io/pyproj/stable/installation.html
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


ModuleNotFoundError: No module named 'geopandas'

In [5]:
def load_taxi_zones(shapefile):
    #load_taxi_zones take a shapefile path str and return a df of such file
    # STR -> DF
    data = gpd.read_file(shapefile)
    return data


In [6]:
data = gpd.read_file("taxi_zones.shp")
data

NameError: name 'gpd' is not defined

In [7]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    #lookup_coords_for_taxi_zone_id take a zone_loc_id and DF to return a tuple of coordinate
    #STR DF -> (Float, Float)
    return None
   

### Calculate Sample Size

In [ ]:
def calculate_sample_size(population):
    raise NotImplementedError()

### Common Functions

The cells below takes the TLC_URL linke to obtain a list of all yellow cab and fhvhv data parquet links.

### get_all_urls_from_tlc_page(taxi_page)

Str_of_URL -> Str_of_html_content

get_all_urls_from_tlc_page takes the TLC_URL link and return its html content for filter_parquet_urls to continue



### filter_parquet_urls(all_urls)

Str_of_html_content -> List_of_parquet_links

filter_parquet_urls takes the html content returned by previous function and return a list of all yellow cabs and fhvhv cars' link

In [56]:
def get_all_urls_from_tlc_page(taxi_page):
    response = requests.get(taxi_page)
    html = response.content
    return html

all_urls = get_all_urls_from_tlc_page(TLC_URL)
all_urls

b'<!DOCTYPE html>\r\n<html>\n<head>\n<META http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<title>TLC Trip Record Data - TLC</title>\n<!--fixed-layout-->\n<!--ls:begin[stylesheet]-->\n<link href="/iwov-resources/fixed-layout/3-Row Simple.css" type="text/css" rel="stylesheet">\n<!--ls:end[stylesheet]-->\n<!--ls:begin[meta-keywords]-->\n<meta name="keywords" content="">\n<!--ls:end[meta-keywords]-->\n<!--ls:begin[meta-description]-->\n<meta name="description" content="">\n<!--ls:end[meta-description]-->\n<!--ls:begin[custom-meta-data]-->\n<!--ls:end[custom-meta-data]-->\n<!--ls:begin[meta-vpath]-->\n<meta name="vpath" content="">\n<!--ls:end[meta-vpath]-->\n<!--ls:begin[meta-page-locale-name]-->\n<meta name="page-locale-name" content="">\n<!--ls:end[meta-page-locale-name]-->\n<!--\n\t\t\t\t\tls:begin[pre-head-injection]\n\t\t\t\t--><!--\n\t\t\t\t\tls:end[pre-head-injection]\n\t\t\t\t--><!--\n\t\t\t\t\tls:begin[social_media_injection]\n\t\t\t\t--><!--\n\t\t\t\t\tls:end[soci

In [57]:
def filter_parquet_urls(all_urls):
    soup = bs4.BeautifulSoup(all_urls, "html.parser")
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    HVFHV_a_tags = soup.find_all("a", attrs={"title": "High Volume For-Hire Vehicle Trip Records"})
    all_a_tags = yellow_a_tags + HVFHV_a_tags
    return [a["href"] for a in all_a_tags]

all_parquet_urls = filter_parquet_urls(all_urls)
all_parquet_urls


['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03

In [6]:
import re

### select_parquet(all_urls)

List_of_parquet -> List_of_parquet

select_parquet(all_urls) takes a list of parquets and filter out all parquets not in the Jan 2020 - Aug 2024 period

In [58]:
def select_parquet(all_urls):
    result =[]
    for i in range(len(all_urls)):
        curr_url = all_urls[i]
        pattern = r"(\d{4})-(\d{2})"
        match = re.search(pattern, curr_url)
        year_str = int(match.group(1))
        month_str = int(match.group(2))
        if (year_str < 2020):
            continue
        if (year_str >= 2024 and month_str > 4):
            continue
        result += [curr_url]       
    return result



In [59]:
required_parquests = select_parquet(all_parquet_urls)
required_parquests

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-07.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-

### select_yellow(urls)

List_of_parquet -> List_of_parquet

select_yellow(urls) takes the parquet filterd by select_parquet and returns all the yellow cabs link as a list

In [37]:
def select_yellow(urls):
    result =[]
    for i in range(len(urls)):
        curr_url = urls[i]
        if not isinstance(curr_url, str):
            continue
        pattern = r"yellow"
        match = re.search(pattern, curr_url)
        if match is None:
            continue
        else: 
            result += [curr_url]       
    return result

In [38]:
required_parquests
yellow_cabs = select_yellow(required_parquests)
yellow_cabs

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-07.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-

### select_fhvhv(urls)

List_of_parquet -> List_of_parquet

Same as select_yellow, but it returns fhvhv links.

In [39]:
def select_fhvhv(urls):
    result =[]
    for i in range(len(urls)):
        curr_url = urls[i]
        if not isinstance(curr_url, str):
            continue
        pattern = r"fhvhv"
        match = re.search(pattern, curr_url)
        if match is None:
            continue
        else: 
            result += [curr_url]       
    return result

In [40]:
required_parquests
fhvhv_cabs = select_fhvhv(required_parquests)
fhvhv_cabs

['https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-03.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-05.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-07.parquet ',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-08.parquet 

### download_one(url, save_name)

Str_of_parquet ->None

Output: 1.message of download result
        2.download a file

download_one(url, save_name) takes a parquet link of yellow cab or fhvhv, then write the data into a local file named save_name. It prints a message if download is successful and raise a http error otherwise

In [61]:
def download_one(url, save_name):
    response = requests.get(url, stream=True)
    response.raise_for_status()  

    with open(save_name, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"File downloaded successfully as {save_name}")
    return None


In [62]:
download_one('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet','fhvhv_tripdata_2024-01.parquet')
df = pd.read_parquet('fhvhv_tripdata_2024-01.parquet')
df

File downloaded successfully as fhvhv_tripdata_2024-01.parquet


,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-01-01 00:21:47,2024-01-01 00:25:06,2024-01-01 00:28:08,2024-01-01 01:05:39,161,158,2.83,...,4.05,2.75,0.0,0.00,40.18,N,N,N,N,N
1,HV0003,B03404,B03404,2024-01-01 00:10:56,2024-01-01 00:11:08,2024-01-01 00:12:53,2024-01-01 00:20:05,137,79,1.57,...,0.89,2.75,0.0,0.00,6.12,N,N,N,N,N
2,HV0003,B03404,B03404,2024-01-01 00:20:04,2024-01-01 00:21:51,2024-01-01 00:23:05,2024-01-01 00:35:16,79,186,1.98,...,1.60,2.75,0.0,0.00,9.47,N,N,N,N,N
3,HV0003,B03404,B03404,2024-01-01 00:35:46,2024-01-01 00:39:59,2024-01-01 00:41:04,2024-01-01 00:56:34,234,148,1.99,...,1.52,2.75,0.0,0.00,11.35,N,N,N,N,N
4,HV0003,B03404,B03404,2024-01-01 00:48:19,2024-01-01 00:56:23,2024-01-01 00:57:21,2024-01-01 01:10:02,148,97,2.65,...,3.43,2.75,0.0,0.00,28.63,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19663925,HV0003,B03404,B03404,2024-01-31 23:24:46,2024-01-31 23:26:11,2024-01-31 23:28:08,2024-01-31 23:32:13,79,113,0.65,...,0.81,2.75,0.0,1.00,5.39,N,N,N,N,N
19663926,HV0003,B03404,B03404,2024-01-31 23:33:02,2024-01-31 23:34:07,2024-01-31 23:34:19,2024-02-01 00:07:53,113,248,13.32,...,3.19,2.75,0.0,0.00,36.43,N,N,N,N,N
19663927,HV0003,B03404,B03404,2024-01-31 23:28:59,2024-01-31 23:30:51,2024-01-31 23:31:14,2024-01-31 23:38:18,161,50,1.31,...,0.89,2.75,0.0,0.00,5.71,N,N,N,N,N
19663928,HV0003,B03404,B03404,2024-01-31 23:39:00,2024-01-31 23:41:03,2024-01-31 23:41:45,2024-01-31 23:52:40,246,163,1.57,...,1.62,2.75,0.0,4.62,8.54,N,N,N,N,N


### data download

The cells below using for loops to call download_one repeatedly to download all yellow cabs and fhvhv parquet data. It uses url.strip() because sometimes we might have a white space in the end of input parquet link which might cause 403 error

In [52]:
for i in fhvhv_cabs:
    url = i
    url = url.strip()
    pattern = r"fhvhv_tripdata_\d{4}-\d{2}\.parquet"
    match = re.search(pattern, url)
    save_name = match.group()
    download_one(url, save_name)

File downloaded successfully as fhvhv_tripdata_2024-01.parquet
File downloaded successfully as fhvhv_tripdata_2024-02.parquet
File downloaded successfully as fhvhv_tripdata_2024-03.parquet
File downloaded successfully as fhvhv_tripdata_2024-04.parquet
File downloaded successfully as fhvhv_tripdata_2023-01.parquet
File downloaded successfully as fhvhv_tripdata_2023-02.parquet
File downloaded successfully as fhvhv_tripdata_2023-03.parquet
File downloaded successfully as fhvhv_tripdata_2023-04.parquet
File downloaded successfully as fhvhv_tripdata_2023-05.parquet
File downloaded successfully as fhvhv_tripdata_2023-06.parquet
File downloaded successfully as fhvhv_tripdata_2023-07.parquet
File downloaded successfully as fhvhv_tripdata_2023-08.parquet
File downloaded successfully as fhvhv_tripdata_2023-09.parquet
File downloaded successfully as fhvhv_tripdata_2023-10.parquet
File downloaded successfully as fhvhv_tripdata_2023-11.parquet
File downloaded successfully as fhvhv_tripdata_2023-12.

In [63]:
for i in yellow_cabs:
    url = i
    url = url.strip()
    pattern = r"yellow_tripdata_\d{4}-\d{2}\.parquet"
    match = re.search(pattern, url)
    save_name = match.group()
    download_one(url, save_name)

File downloaded successfully as yellow_tripdata_2024-01.parquet
File downloaded successfully as yellow_tripdata_2024-02.parquet
File downloaded successfully as yellow_tripdata_2024-03.parquet
File downloaded successfully as yellow_tripdata_2024-04.parquet
File downloaded successfully as yellow_tripdata_2023-01.parquet
File downloaded successfully as yellow_tripdata_2023-02.parquet
File downloaded successfully as yellow_tripdata_2023-03.parquet
File downloaded successfully as yellow_tripdata_2023-04.parquet
File downloaded successfully as yellow_tripdata_2023-05.parquet
File downloaded successfully as yellow_tripdata_2023-06.parquet
File downloaded successfully as yellow_tripdata_2023-07.parquet
File downloaded successfully as yellow_tripdata_2023-08.parquet
File downloaded successfully as yellow_tripdata_2023-09.parquet
File downloaded successfully as yellow_tripdata_2023-10.parquet
File downloaded successfully as yellow_tripdata_2023-11.parquet
File downloaded successfully as yellow_t

### Process Taxi Data

In [ ]:
def get_and_clean_taxi_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [ ]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.contact(all_uber_dataframes)
    return uber_data

In [ ]:
def load_and_clean_uber_data():
    raise NotImplementedError()

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)